# About

This notebook reads a file from the file system written in Python and converts it into C++

In [1]:
import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import subprocess

In [2]:
load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
if not OPENAI_API_KEY:
    print("OPENAI_API_KEY not loaded.")
    sys.exit(1)

GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
if not GEMINI_API_KEY:
    print("GEMINI_API_KEY not loaded.")

ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY')
if not ANTHROPIC_API_KEY:
    print("ANTHROPIC_API_KEY not loaded.")

In [3]:
# Init
openai = OpenAI()
gemini = google.generativeai.configure()
claude = anthropic.Anthropic()

OPENAI_MODEL = "gpt-4o"
GEMINI_MODEL = "gemini-2.5-flash"
CLAUDE_MODEL = "claude-3-7-sonnet-20250219"

In [4]:
system_prompt = """
You are a coding assistant that reimplements Python code in high performance C++ code for a low spec laptop.
You must follow the next guidelines:
- Do not output anything else other than the translated C++ code
- Do not provide any explanation or thoughts
- Add meaningful but short comments in the code
- The C++ output code will have to produce an identical output with the Python code
- The C++ output code will have to be efficient and as fast as possible
- The C++ output code will have to include all necessary C++ packages such as iostream
- The C++ output code will have to produce identical output with the Python code
- Pay attention to number types to ensure no numeric overflows when converting
- The Python code will always be included between triple ticks ``` and a py markdown notation
- The C++ output code will have to be enclosed between triple ticks ``` and a cpp markdown notation
"""

In [5]:
def user_prompt(python_script):
    user_prompt = f"""
    Rewrite the following Python code in C++ according to your instructions:
    ```py
    {python_script}
    ```
    """
    return user_prompt

In [6]:
def write_output(cpp_script_name, cpp_script):
    with open(f"{os.path.splitext(cpp_script_name)[0]}.cpp", "w") as f:
        f.write(cpp_script.replace("```cpp", "").replace("```", ""))

In [29]:
script_dir = "resources/"
python_script_name = f"{script_dir}pi_calculator.py"
cpp_script_name_gpt = f"{script_dir}pi_calculator_gpt.cpp"
cpp_script_name_claude = f"{script_dir}pi_calculator_claude.cpp"

with open(python_script_name, 'r') as f:
    python_script = f.read()

!cat resources/pi_calculator.py

#!/usr/bin/env python3
# This is a pi calculation script
import time
import argparse

def calculate(iterations, p1, p2):
  result = 1.0
  for i in range(1, iterations + 1):
    j = i *  p1 - p2
    result -= (1/j)
    j = i * p1 + p2
    result += (1/j)
  return result

def main():
  parser = argparse.ArgumentParser(description="Calculate a mathematical series.")
  parser.add_argument('--iterations', type=int, default=100000000, help='Number of iterations to perform')
  args = parser.parse_args()

  # Use the provided iterations value
  iterations = args.iterations
  start_time = time.time()
  result = calculate(iterations, 4, 1) * 4
  end_time = time.time()
  print(f"Result: {result:.12f}")
  print(f"Execution time: {end_time - start_time:.6f} seconds")

if __name__ == "__main__":
  main()


In [11]:
def gpt_convert(python_script):
    response = openai.chat.completions.create(
        model = OPENAI_MODEL,
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt(python_script) }
        ]
    )
    return response.choices[0].message.content

In [15]:
gpt_output = gpt_convert(python_script)

In [41]:
def claude_convert(python_script):
    message = claude.messages.create(
        model = "claude-3-7-sonnet-20250219",
        max_tokens = 1000,
        temperature = 0.7,
        system = system_prompt,
        messages = [
            { "role": "user", "content": user_prompt(python_script) }
        ]
    )
    return message.content[0].text

In [42]:
claude_output = claude_convert(python_script)

In [30]:
print(gpt_output)


```cpp
#include <iostream>
#include <chrono>
#include <cmath>

// Function to calculate the series using iterations, p1, and p2
double calculate(int iterations, int p1, int p2) {
    double result = 1.0;
    for (int i = 1; i <= iterations; ++i) {
        double j = i * p1 - p2;
        result -= (1.0 / j);
        j = i * p1 + p2;
        result += (1.0 / j);
    }
    return result;
}

int main(int argc, char *argv[]) {
    int iterations = 100000000; // Default number of iterations
    if (argc > 1) {
        iterations = std::stoi(argv[1]); // Read iterations from command line arguments if provided
    }

    auto start_time = std::chrono::high_resolution_clock::now(); // Start timing
    double result = calculate(iterations, 4, 1) * 4; // Call calculation function
    auto end_time = std::chrono::high_resolution_clock::now(); // End timing
    std::chrono::duration<double> elapsed = end_time - start_time; // Calculate elapsed time

    std::cout.precision(12);
    std::cout << "R

In [43]:
print(claude_output)

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>
#include <cstdlib>
#include <string>

// This is a pi calculation script
double calculate(int iterations, int p1, int p2) {
    double result = 1.0;
    for (int i = 1; i <= iterations; ++i) {
        double j = i * p1 - p2;
        result -= (1.0/j);
        j = i * p1 + p2;
        result += (1.0/j);
    }
    return result;
}

int main(int argc, char* argv[]) {
    // Default iterations
    int iterations = 100000000;
    
    // Parse command line arguments
    for (int i = 1; i < argc; ++i) {
        std::string arg = argv[i];
        if (arg == "--iterations" && i + 1 < argc) {
            iterations = std::atoi(argv[i + 1]);
            i++; // Skip the next argument
        }
    }
    
    // Measure execution time
    auto start_time = std::chrono::high_resolution_clock::now();
    double result = calculate(iterations, 4, 1) * 4;
    auto end_time = std::chrono::high_resolution_clock::now();
    
    // Calculate

In [32]:
write_output(cpp_script_name_gpt, gpt_output)

!cat resources/pi_calculator_gpt.cpp



#include <iostream>
#include <chrono>
#include <cmath>

// Function to calculate the series using iterations, p1, and p2
double calculate(int iterations, int p1, int p2) {
    double result = 1.0;
    for (int i = 1; i <= iterations; ++i) {
        double j = i * p1 - p2;
        result -= (1.0 / j);
        j = i * p1 + p2;
        result += (1.0 / j);
    }
    return result;
}

int main(int argc, char *argv[]) {
    int iterations = 100000000; // Default number of iterations
    if (argc > 1) {
        iterations = std::stoi(argv[1]); // Read iterations from command line arguments if provided
    }

    auto start_time = std::chrono::high_resolution_clock::now(); // Start timing
    double result = calculate(iterations, 4, 1) * 4; // Call calculation function
    auto end_time = std::chrono::high_resolution_clock::now(); // End timing
    std::chrono::duration<double> elapsed = end_time - start_time; // Calculate elapsed time

    std::cout.precision(12);
    std::cout << "Result:

In [44]:
write_output(cpp_script_name_claude, claude_output)

!cat resources/pi_calculator_claude.cpp


#include <iostream>
#include <iomanip>
#include <chrono>
#include <cstdlib>
#include <string>

// This is a pi calculation script
double calculate(int iterations, int p1, int p2) {
    double result = 1.0;
    for (int i = 1; i <= iterations; ++i) {
        double j = i * p1 - p2;
        result -= (1.0/j);
        j = i * p1 + p2;
        result += (1.0/j);
    }
    return result;
}

int main(int argc, char* argv[]) {
    // Default iterations
    int iterations = 100000000;
    
    // Parse command line arguments
    for (int i = 1; i < argc; ++i) {
        std::string arg = argv[i];
        if (arg == "--iterations" && i + 1 < argc) {
            iterations = std::atoi(argv[i + 1]);
            i++; // Skip the next argument
        }
    }
    
    // Measure execution time
    auto start_time = std::chrono::high_resolution_clock::now();
    double result = calculate(iterations, 4, 1) * 4;
    auto end_time = std::chrono::high_resolution_clock::now();
    
    // Calculate durat

# Compare original with produced script

Note: Make sure to have `g++` installed on your system.

In [45]:
!python resources/pi_calculator.py

Result: 3.141592658589
Execution time: 6.735307 seconds


In [46]:
!g++ -o resources/pi_calculator_gpt resources/pi_calculator_gpt.cpp
!resources/pi_calculator_gpt

Result: 3.14159265859
Execution time: 0.423047741 seconds


In [47]:
!g++ -o resources/pi_calculator_claude resources/pi_calculator_claude.cpp
!resources/pi_calculator_claude

Result: 3.141592658589
Execution time: 0.427392 seconds
